# Load the data

In [ ]:
# Initial Python environment setup...
import numpy as np # linear algebra
import pandas as pd # CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
train_df =  pd.read_csv('../input/train.csv', nrows = 5_000_000)

In [ ]:
def haversine_np(lon1, lat1, lon2, lat2):
    """
    https://stackoverflow.com/questions/29545704/fast-haversine-approximation-python-pandas
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)

    All args must be of equal length.    

    """
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2
    c = 2 * np.arcsin(np.sqrt(a))
    km = 6367 * c
    return km

In [ ]:
train_df['distance'] = haversine_np(train_df['pickup_longitude'], train_df['pickup_latitude'], 
                                    train_df['dropoff_longitude'], train_df['dropoff_latitude'])

In [ ]:
train_df['pickup_datetime'] = pd.to_datetime(train_df['pickup_datetime']) 

In [ ]:
train_df['year'] = train_df['pickup_datetime'].dt.year
train_df['month'] = train_df['pickup_datetime'].dt.month
train_df['day'] = train_df['pickup_datetime'].dt.day
train_df['hour'] = train_df['pickup_datetime'].dt.hour
train_df['minute'] = train_df['pickup_datetime'].dt.minute

### Tidy data

In [ ]:
print('Old size: %d' % len(train_df))
train_df = train_df.dropna(how = 'any', axis = 'rows')
print('New size: %d' % len(train_df))

In [ ]:
new_york_lat = 40
new_york_long = -74
train_df.describe()

Lets remove the data if it is more than 5 off from the mean

In [ ]:
cond = True
for col in {'pickup_latitude', 'pickup_longitude', 'dropoff_latitude', 'dropoff_longitude'}:
    cond &= abs(train_df[col] - train_df[col].mean()) < 5


In [ ]:
print('Old size: %d' % len(train_df))
train_df = train_df[cond]
print('New size: %d' % len(train_df))

In [ ]:
train_df.describe()

# Try to identify top 10 most common pickup an drop off locations

In [ ]:
for col in {'pickup_latitude', 'pickup_longitude', 'dropoff_latitude', 'dropoff_longitude'}:
    train_df['rough' + col] = train_df[col].round(2)

In [ ]:
a=train_df.groupby(['roughpickup_latitude','roughpickup_longitude'])[['pickup_latitude', 'pickup_longitude']].agg(['mean','count'])

In [ ]:
a.columns = ['mean_pickup_latitude','c1','mean_pickup_longitude','c2']
a.head()
a.sort_values(['c1','c2'],ascending=False).head()
# this is the pensylvania station

In [ ]:
b=train_df.groupby(['roughdropoff_latitude','roughdropoff_longitude'])[['dropoff_latitude', 'dropoff_longitude']].agg(['mean','count'])

In [ ]:
b.columns = ['mean_dropoff_latitude','c1','mean_dropoff_longitude','c2']
b.head()
b.sort_values(['c1','c2'],ascending=False).head(n=20)
# this is near a hotel also near the station?

In [ ]:
# what about the airports?
# jfk 40.6413151,-73.7803278,
b[ b['mean_dropoff_latitude']<40.7].sort_values(['c1','c2'],ascending=False).head()


In [ ]:
b['c1'].plot.hist()
plt.title('occurance of counts of rough dropoff locations')
plt.yscale('log')
# this figure is showing us that while most locations have hardly any unique dropoffs, quitea few places are getting
# loads of counts

In [ ]:
a['c1'].plot.hist()
plt.title('occurance of counts of rough dropoff locations')
plt.yscale('log')

In [ ]:
a[a['c1']>1000].shape
a.shape

### busy-ness metric of a location?
I was intending on doing something else, like maybe marking the top 1000 or top 10 pickup/dropoff locations, then looking at distances from each one of these. Instead, A very quick parameter I can do is use the count parameter as an estimate of how busy that location is!


In [ ]:
a = a[['c1']].reset_index().rename(columns={'c1':'pickup_busyness'})
b = b[['c1']].reset_index().rename(columns={'c1':'dropoff_busyness'})

In [ ]:
train_df = pd.merge(train_df,a, how='left')
train_df = pd.merge(train_df,b, how='left')

In [ ]:
train_df.head()

### Train our model


In [ ]:
X = train_df[['distance','year','month','day','hour','pickup_busyness','dropoff_busyness']].values
Y = train_df['fare_amount'].values

## What about some random forests

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
# from hyper parameter tuning:
kwargs = {'bootstrap': True,
 'max_depth': None,
 'max_features': 3,
 'min_samples_leaf': 9,
 'min_samples_split': 2}
rand_regr = RandomForestRegressor(n_estimators=20, **kwargs)

In [ ]:
rand_regr.fit(X, Y)

In [ ]:
y_pred = rand_regr.predict(X)
print('chi squared  rand forest with date %s' % (np.sum((Y-y_pred)**2.)/len(Y))**0.5)

In [ ]:
rand_regr.score(X,Y)

# Cross validation of hyperparamter tuning
 see http://scikit-learn.org/stable/auto_examples/model_selection/plot_randomized_search.html#sphx-glr-auto-examples-model-selection-plot-randomized-search-py
 
 

In [ ]:
# from sklearn.model_selection import RandomizedSearchCV
# from time import time
# from scipy.stats import randint as sp_randint

In [ ]:
# build a classifier
# clf = RandomForestRegressor(n_estimators=20)

In [ ]:
# Utility function to report best scores
# def report(results, n_top=3):
#     for i in range(1, n_top + 1):
#         candidates = np.flatnonzero(results['rank_test_score'] == i)
#         for candidate in candidates:
#             print("Model with rank: {0}".format(i))
#             print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
#                   results['mean_test_score'][candidate],
#                   results['std_test_score'][candidate]))
#             print("Parameters: {0}".format(results['params'][candidate]))
#             print("")


# # specify parameters and distributions to sample from
# param_dist = {"max_depth": [3, None],
#               "max_features": sp_randint(1, 6),
#               "min_samples_split": sp_randint(2, 11),
#               "min_samples_leaf": sp_randint(1, 11),
#               "bootstrap": [True, False],
# #               "criterion": ["gini", "entropy"]
#              }

# # run randomized search
# n_iter_search = 20
# random_search = RandomizedSearchCV(clf, param_distributions=param_dist,
#                                    n_iter=n_iter_search, cv=3)

# start = time()
# random_search.fit(X, Y)
# # print("RandomizedSearchCV took %.2f seconds for %d candidates"
# #       " parameter settings." % ((time() - start), n_iter_search))
# # report(random_search.cv_results_)

In [ ]:
# pd.DataFrame(random_search.cv_results_)

In [ ]:
# random_search.best_estimator_

In [ ]:
# random_search.best_params_

In [ ]:
# random_search.best_score_

# Look into learning curve
I remember from the machine learning course on coursera that you must look at the learning curve of the algorithm to see what it really needs. Does it need more data, more features? Hopefully the learning curve will tell

In [ ]:
# from sklearn.model_selection import learning_curve

In [ ]:
# train_sizes, train_scores, test_scores = learning_curve(RandomForestRegressor(), X, Y)

In [ ]:
# train_scores_mean = np.mean(train_scores, axis=1)
# train_scores_std = np.std(train_scores, axis=1)
# test_scores_mean = np.mean(test_scores, axis=1)
# test_scores_std = np.std(test_scores, axis=1)
# plt.grid()

# plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
#                  train_scores_mean + train_scores_std, alpha=0.1,
#                  color="r")
# plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
#                  test_scores_mean + test_scores_std, alpha=0.1, color="g")
# plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
#          label="Training score")
# plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
#          label="Cross-validation score")


# plt.legend(loc="best")

# TODO
usefull link: https://www.coursera.org/learn/machine-learning/lecture/Kont7/learning-curves

I think the figure above shows two things: I need to tune the parameters of the random forest (regularisation for example). And, getting more data might help!

Lets plot the learning curve after we have fixed some regulirazation terms.




Random forest seems like a good fit! maybe do a submission?

In [ ]:
test_df =  pd.read_csv('../input/test.csv')

In [ ]:
test_df['distance'] = haversine_np(test_df['pickup_longitude'], test_df['pickup_latitude'], 
                                    test_df['dropoff_longitude'], test_df['dropoff_latitude'])

In [ ]:
test_df['pickup_datetime'] = pd.to_datetime(test_df['pickup_datetime']) 

In [ ]:
test_df['year'] = test_df['pickup_datetime'].dt.year
test_df['month'] = test_df['pickup_datetime'].dt.month
test_df['day'] = test_df['pickup_datetime'].dt.day
test_df['hour'] = test_df['pickup_datetime'].dt.hour
test_df['minute'] = test_df['pickup_datetime'].dt.minute

In [ ]:
for col in {'pickup_latitude', 'pickup_longitude', 'dropoff_latitude', 'dropoff_longitude'}:
    test_df['rough' + col] = test_df[col].round(2)

In [ ]:
test_df = pd.merge(test_df,a, how='left')
test_df = pd.merge(test_df,b, how='left')

In [ ]:
test_df['pickup_busyness'] = test_df['pickup_busyness'].fillna(1)
test_df['dropoff_busyness'] = test_df['dropoff_busyness'].fillna(1)

In [ ]:
X_to_pred = test_df[['distance','year','month','day','hour','pickup_busyness','dropoff_busyness']].values
y_pred = rand_regr.predict(X_to_pred)

In [ ]:
submission = pd.DataFrame(
    {'key': test_df.key, 'fare_amount': y_pred},
    columns = ['key', 'fare_amount'])
submission.to_csv('submission.csv', index = False)

In [ ]:
# x=np.arange(0,500)
# y=regr.predict(x.reshape(-1,1))
# plt.plot(X,Y,'o')
# plt.plot(x,y)
# plt.show()

look! No wonder our fit is bad! There is this nice linear bit in the middle, then there is all these its to the right and left

In [ ]:
# with sns.axes_style("white"):
#     sns.jointplot(x=X.flatten(), y=Y, kind="hex", color="k", bins='log');

In [ ]:
# X = X.flatten()
# mask = (X <50) & (Y <100)
# # mask
# # fig, ax = plt.subplots()

# with sns.axes_style("white"):
#     p = sns.jointplot(x=X[mask], y=Y[mask], kind="hex", color="k", bins='log');

# x=np.arange(0,50)
# y=regr.predict(x.reshape(-1,1))
# p.ax_joint.plot(x,y)

We can see here that the linear model works for the low distance scenario, but for distance greater than 10, it is no longer a good fit. A quadratic term might do better!

Maybe do a KNN to see if I can find interesting features?

In [ ]:
# from sklearn.cluster import KMeans

In [ ]:
# X_clus = train_df[['fare_amount','distance']].values

In [ ]:
# km = KMeans(n_clusters=5,random_state=0)

In [ ]:
# km.fit(X_clus)

In [ ]:
# plt.scatter(X_clus[:,0],X_clus[:,1], c=km.predict(X_clus))
# plt.show()

In [ ]:
# from sklearn.cluster import MiniBatchKMeans

In [ ]:
# kmm = MiniBatchKMeans(n_clusters=5, random_state=0)

In [ ]:
# kmm.fit(X_clus)

In [ ]:
# plt.scatter(X_clus[:,0],X_clus[:,1], c=kmm.predict(X_clus))
# plt.show()

In [ ]:
# plt.hist(X_clus[:,0]/(X_clus[:,1]+1e-10),bins=100,range=[0,60])
# plt.show()


In [ ]:
# from sklearn.cluster import DBSCAN

In [ ]:
# dbs = DBSCAN(eps=0.3, min_samples=10)

In [ ]:
# dbs.fit(X_clus)

In [ ]:
# plt.scatter(X_clus[:,0],X_clus[:,1], c=dbs.predict(X_clus))
# plt.show()